# Homework 3

Philip Wallis

PTW190000

CS 4371.001

## Question 1

In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

# Read the file into an RDD
rdd = sc.textFile("dbfs:/FileStore/shared_uploads/ptw190000@utdallas.edu/input_hw1.txt")

# Tokenize and preprocess
tokenized_rdd = rdd.flatMap(lambda line: line.split(" ")) \
                   .filter(lambda word: word != "") \
                   .map(lambda word: word.lower().strip('.,!?'))

In [ ]:
# 1A. Count occurrences
word_count = tokenized_rdd.countByValue()
display(word_count)

defaultdict(int,
            {'history': 145,
             'of': 15449,
             'the': 28969,
             'united': 507,
             'states': 768,
             'by': 2655,
             'charles': 29,
             'a': 6514,
             'beard': 11,
             'and': 11608,
             'mary': 87,
             'r': 8,
             'new': 776,
             'york': 187,
             'macmillan': 2,
             'company': 101,
             '1921': 7,
             '_all': 1,
             'rights': 129,
             'reserved_': 1,
             'copyright': 14,
             'set': 119,
             'up': 591,
             'electrotyped': 1,
             'published': 16,
             'march': 71,
             'norwood': 2,
             'press': 44,
             'j.s': 5,
             'cushing': 2,
             'co.--berwick': 1,
             '&': 3,
             'smith': 11,
             'co': 2,
             'mass': 31,
             'u.s.a': 1,
             'preface': 2,
       

In [ ]:
# 1B. Find Specific Words
specific_words = ["america", "president", "washington"]
specific_word_count = {word: word_count[word] for word in specific_words if word in word_count}
display(specific_word_count)

{'america': 233, 'president': 315, 'washington': 141}

In [ ]:
# 1C. Top 10 Words
import operator

# Sort by count in descending order and take the top 10
sorted_word_count = sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)[:10]
display(sorted_word_count)

_1,_2
the,28969
of,15449
and,11608
to,9148
in,8083
a,6514
was,3927
that,3014
he,2728
with,2705


In [ ]:
# imports for problem 2
temperature_rdd = sc.textFile("dbfs:/FileStore/shared_uploads/ptw190000@utdallas.edu/city_temperature.csv")
country_rdd = sc.textFile("dbfs:/FileStore/shared_uploads/ptw190000@utdallas.edu/country_list.csv")

# headers
header_temperature = temperature_rdd.first()
header_country = country_rdd.first()

# remove headers and quotes
temperature_rdd = temperature_rdd.filter(lambda line: line != header_temperature)
country_rdd = country_rdd.filter(lambda line: line != header_country).map(lambda x: x.replace("\"", ""))

## Question 2

In [ ]:
# 2A. Average Temperature by Region

def average_by_region(rdd):
    #clean
    region_temp = (rdd.map(lambda x: x.split(","))
                     .filter(lambda x: len(x) >= 8 and x[7] != "-99")
                     .map(lambda x: (x[0], float(x[7]))))
    
    #join
    region_temp_count = region_temp.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
    region_avg_temp = region_temp_count.mapValues(lambda x: x[0] / x[1])
    return region_avg_temp

region_avg_temperature = average_by_region(temperature_rdd)

for region, avg_temp in region_avg_temperature.collect():
    print(f"{region}\t{avg_temp}")


Asia	68.10972259874494
Africa	74.40260231125495
Australia/South Pacific	62.30369323842191
Europe	51.94717142841652
Middle East	73.84068255374054
North America	56.15019771858212
South/Central America & Carribean	72.20202379397209


In [ ]:
# 2B. Average by Year in Asia
def average_by_year_for_each_country_in_asia(rdd):
    # clean, for asia
    asia_temp = (rdd.map(lambda x: x.split(","))
                   .filter(lambda x: len(x) >= 8 and x[0] == "Asia" and x[7] != "-99")
                   .map(lambda x: ((x[1], x[6]), float(x[7]))))  # Composite key with country and year
    
    # average by year for each country
    asia_temp_count = asia_temp.mapValues(lambda x: (x, 1))
    asia_temp_count_reduced = asia_temp_count.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
    asia_avg_temp = asia_temp_count_reduced.mapValues(lambda x: x[0] / x[1])
    
    return asia_avg_temp

asia_avg_temp_by_year_and_country = average_by_year_for_each_country_in_asia(temperature_rdd)

for (country, year), avg_temp in asia_avg_temp_by_year_and_country.collect():
    print(f"{country}, {year} - {avg_temp}")

Bangladesh, 1996 - 78.76584507042256
Bangladesh, 1997 - 77.15481927710844
Bangladesh, 1998 - 79.28705882352948
Bangladesh, 1999 - 77.08787878787875
Bangladesh, 2000 - 79.0985507246377
Bangladesh, 2001 - 78.95656934306568
Bangladesh, 2005 - 77.85701754385963
Bangladesh, 2006 - 79.3555944055944
Bangladesh, 2008 - 75.64156626506025
Bangladesh, 2009 - 79.02384341637014
China, 1995 - 59.772493150684916
China, 2002 - 60.71225055432367
China, 2003 - 60.6517827756445
China, 2004 - 60.931803278688555
China, 2007 - 61.71794166208038
China, 2010 - 59.66734246575344
China, 2011 - 59.932986301369965
China, 2012 - 59.583989071038275
China, 2013 - 60.763013698630054
China, 2014 - 61.4255085211654
China, 2015 - 61.68868131868132
China, 2016 - 61.59270315091207
China, 2017 - 62.01704109589035
China, 2019 - 61.992033426183895
China, 2020 - 52.202391629297395
Hong Kong, 1996 - 74.97103825136611
Hong Kong, 1997 - 75.2531506849315
Hong Kong, 1998 - 76.2457063711912
Hong Kong, 1999 - 75.53846153846156
Hong 

In [ ]:
# 2C. Average Temperature in Spain by City
def average_temperature_by_city_in_spain(rdd):
    # Filter out invalid entries and focus on Spain
    filtered_rdd = rdd.map(lambda x: x.split(",")) \
                      .filter(lambda x: len(x) >= 8) \
                      .filter(lambda x: x[7] != "-99") \
                      .filter(lambda x: x[1] == "Spain")
    
    # Map city to its temperature, and count
    city_temperature = filtered_rdd.map(lambda x: (x[3], (float(x[7]), 1)))
    
    # Reduce to get sum and count for each city, then compute the average
    sum_count_rdd = city_temperature.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
    average_by_city_rdd = sum_count_rdd.mapValues(lambda x: x[0] / x[1])
    
    return average_by_city_rdd

average_temperature_by_city_in_spain_rdd = average_temperature_by_city_in_spain(temperature_rdd)

# Print the result
for city, avg_temp in average_temperature_by_city_in_spain_rdd.collect():
    print(f"{city}\t{avg_temp}")

Barcelona	61.78984408835005
Bilbao	59.173121887854684
Madrid	58.91773111062988


In [ ]:
# 2D: For each country, find the capital and average of AvgTemperature of that capital city.
def average_temperature_by_capital(rdd_temp, rdd_country):
    country_capital = rdd_country.map(lambda x: x.split(",")).map(lambda x: (x[0].strip(), x[1].strip()))
    city_temp = rdd_temp.map(lambda x: x.split(",")).filter(lambda x: x[7] != "-99").map(lambda x: ((x[1].strip(), x[3].strip()), float(x[7])))
    joined = city_temp.join(country_capital.map(lambda x: ((x[0], x[1]), None)))
    aggregated = joined.mapValues(lambda x: x[0]).aggregateByKey((0, 0), 
                                                                  lambda acc, value: (acc[0] + value, acc[1] + 1),
                                                                  lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]))
    avg_temp_by_capital = aggregated.mapValues(lambda x: x[0] / x[1])
    return avg_temp_by_capital

avg_temp_by_capital = average_temperature_by_capital(temperature_rdd, country_rdd)
for (country, capital), avg_temp in avg_temp_by_capital.collect():
    print(f"{country} {capital}\t{avg_temp}")


Morocco Rabat	63.38186152345862
Mozambique Maputo	74.74497458410396
Senegal Dakar	76.33087262872628
Sierra Leone Freetown	81.71703655352484
Tunisia Tunis	67.08807409012111
Japan Tokyo	61.566027931146316
Singapore Singapore	82.24119748809011
Uzbekistan Tashkent	60.52799476325585
Austria Vienna	51.46943722943731
Finland Helsinki	42.672104351591095
Hungary Budapest	51.57030421132389
Poland Warsaw	48.25283672585513
Portugal Lisbon	62.625254825417265
Romania Bucharest	52.83972068853537
Russia Moscow	42.39905793178134
Spain Madrid	58.91773111062988
Qatar Doha	82.96221692491069
Turkey Ankara	51.39081853616294
Cuba Havana	75.84939520562996
Guatemala Guatemala City	66.8639035591273
Algeria Algiers	64.37253818654507
Gambia Banjul	79.01938748685622
Mauritania Nouakchott	78.11336918000707
Namibia Windhoek	69.32590835454769
Indonesia Jakarta	83.36622922134751
Kyrgyzstan Bishkek	53.47416841844964
Laos Vientiane	81.05564603691633
North Korea Pyongyang	51.873763964163494
South Korea Seoul	53.404299328